In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='/home/reflex/tmp', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='/home/reflex/tmp', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
"""
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
"""

[1,  2000] loss: 2.225
[1,  4000] loss: 1.896
[1,  6000] loss: 1.704
[1,  8000] loss: 1.602
[1, 10000] loss: 1.528
[1, 12000] loss: 1.453
[2,  2000] loss: 1.391
[2,  4000] loss: 1.384
[2,  6000] loss: 1.344
[2,  8000] loss: 1.337
[2, 10000] loss: 1.292
[2, 12000] loss: 1.295


NameError: name 'printb' is not defined

## + ResNet

In [6]:
pretrained_resnet18 = models.resnet18(pretrained=True, num_classes=10, input_channels=1)
resnet = nn.Sequential(*list(pretrained_resnet18.children())) 

TypeError: __init__() got an unexpected keyword argument 'input_channels'

In [5]:
resnet

NameError: name 'resnet' is not defined

In [15]:
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        self.net = resnet
        self.fc1 = nn.Linear(1000, 512) # + vector size!
        self.fc2 = nn.Linear(512, 7) 
        self.sigm = nn.Sigmoid()
        
    def forward(self, x):
        out = self.net(x)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sigm()
        return out

testNet = TestNet()
#pNet.to(device)
print("OK!")

OK!


In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(testNet.parameters(), lr=0.001, momentum=0.9)

In [17]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = testNet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: Given input size: (512x1x1). Calculated output size: (512x-5x-5). Output size is too small at /opt/conda/conda-bld/pytorch_1532579245307/work/aten/src/THNN/generic/SpatialAveragePooling.c:64

### + Fast.ai

In [25]:
pretrained_resnet34 = models.resnet34(pretrained=True)

In [14]:
pretrained_resnet34

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [102]:
resnet_1 = nn.Sequential(*list(pretrained_resnet34.children())[:-1]) 
resnet_2 = nn.Sequential(*list(pretrained_resnet34.children())[:-1]) 

norm_model_loc = "/home/reflex/refleX/results/fastai_experiments/first_search_overview/models/best_resnet34_imgsize512_batch8_unfreeze.h5"
resnet_1.load_state_dict(torch.load(norm_model_loc), strict=False)

# Freeze weights
for param in resnet_1.parameters():
    param.requires_grad = False
    
polar_model_loc = "/home/reflex/refleX/results/fastai_experiments/13_11_polar_sample/models/best_resnet34_imgsize512_batch32_unfreeze.h5"
resnet_2.load_state_dict(torch.load(polar_model_loc), strict=False)

# Freeze weights
for param in resnet_2.parameters():
    param.requires_grad = False
    
class ParallelNet(nn.Module):
    def __init__(self):
        super(ParallelNet, self).__init__()
        self.net_1 = resnet_1
        self.net_2 = resnet_2
        self.fc1 = nn.Linear(1024, 512) # + vector size!
        self.fc2 = nn.Linear(512, 7) 
        self.sigm = nn.Sigmoid()
        
    def forward(self, x):
        # input: v - vector
        x_1 = self.net_1(x)
        x_2 = self.net_2(x)
        out = torch.cat((x_1, x_2), 1)
        
        # out = torch.cat((x_1, x_2, v), 1) ?
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sigm()
        return out

pNet = ParallelNet()
#pNet.to(device)
print("OK!")

OK!


In [16]:
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(pNet.parameters(), lr=0.001, momentum=0.9)

In [17]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.195
[1,  4000] loss: 1.189
[1,  6000] loss: 1.208
[1,  8000] loss: 1.217
[1, 10000] loss: 1.188
[1, 12000] loss: 1.186
[2,  2000] loss: 1.190
[2,  4000] loss: 1.209
[2,  6000] loss: 1.206
[2,  8000] loss: 1.191
[2, 10000] loss: 1.170
[2, 12000] loss: 1.200
Finished Training


In [68]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
import os
from skimage import io

In [148]:
reflex_csv = '/home/reflex/refleX/metadata/csv/train.csv'
reflex_images = '/home/reflex/refleX/metadata/labeled/original512'

In [1]:
class reflexDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=transforms.ToTensor()):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.reflex_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.reflex_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.reflex_frame.iloc[idx, 0])
        image = io.imread(img_name + ".png")
        labels = self.reflex_frame.iloc[idx, 1:].values
        labels = labels.astype('float').reshape(-1)
        sample = [image, labels]

        if self.transform:
            sample = self.transform(sample)

        return sample

NameError: name 'Dataset' is not defined

In [150]:
reflex_ds = reflexDataset(csv_file=reflex_csv, root_dir=reflex_images, transform=transforms.Compose([ToTensor()]))

In [151]:
dataset_loader = torch.utils.data.DataLoader(reflex_ds, batch_size=4, shuffle=True, num_workers=4)

In [152]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(pNet.parameters(), lr=0.001, momentum=0.9)

In [153]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataset_loader, 0):
        
        # get the inputs
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = pNet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

AttributeError: Traceback (most recent call last):
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-143-969d32098be4>", line 23, in __getitem__
    image = image.type('torch.DoubleTensor')
AttributeError: 'numpy.ndarray' object has no attribute 'type'


Process Process-100:
Process Process-102:
Process Process-99:
Process Process-101:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
Traceback (most recent call last):
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/reflex/miniconda3/envs/torch/lib/python3.6/multi